## Kaggle access

To login to Kaggle, you can either store your `kaggle.json` credentials file at
`~/.kaggle/kaggle.json` or run the following in a Colab environment. See the
[`kagglehub` package documentation](https://github.com/Kaggle/kagglehub#authenticate)
for more details.

In [1]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


## Install dependencies

In [2]:
!pip install -q -U torch immutabledict sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## Download model weights

In [3]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']

In [5]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

Attaching model 'google/gemma/pyTorch/2b-it' to your Colab notebook...


## Download the model implementation

In [6]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 71 (delta 34), reused 53 (delta 20), pack-reused 0
Receiving objects: 100% (71/71), 2.02 MiB | 5.52 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [7]:
import sys

sys.path.append('gemma_pytorch')

In [8]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

## Setup the model

In [9]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

# Experiment Constants

In [35]:
example_lease = """Apartment Lease Agreement

This Lease Agreement ("Agreement") is entered into on February 24, 2024, by and between John Doe ("Landlord"), and Jane Smith and Alex Johnson ("Tenant").

Premises: The Landlord agrees to lease the premises located at 123 Maple Street, Apartment 5B, Springfield, IL, ("Premises"), to the Tenant.

Term: The lease begins on March 1, 2024, and ends on February 28, 2025 ("Lease Term").

Rent: The Tenant agrees to pay the Landlord $1,200 per month. Rent is due on the first day of each month. Late payments will incur a fee of $50 after a grace period of 5 days.

Security Deposit: The Tenant shall pay a security deposit of $2,400 before moving in, to cover any damages or unpaid rent. The deposit will be returned within 30 days after the lease ends, minus any deductions for repairs or unpaid rent.

Utilities: Tenant is responsible for paying all utility bills, including electricity, gas, water, and internet, unless otherwise specified.

Occupants: The apartment is to be occupied strictly by Jane Smith and Alex Johnson. No additional occupants may reside on the Premises without prior written consent from the Landlord.

Pets: Pets are allowed with a $300 deposit. Specific breeds, sizes, and numbers of pets are subject to restrictions.

Maintenance and Repairs: The Tenant is responsible for keeping the Premises in a clean and sanitary condition. The Landlord is responsible for major repairs and maintenance of the building's structural elements.

Subletting: The Tenant shall not sublet the Premises or assign this lease without the Landlord’s written consent.

Noise and Conduct: Tenants agree to not cause or allow any noise or activity on the Premises that might disturb the peace and quiet of another tenant or neighbor.

Smoking: Smoking is not allowed within the apartment and building premises.

Termination: Either party may terminate the lease with a 30-day written notice before the lease term ends. Early termination fees may apply.

Governing Law: This Agreement shall be governed by the laws of the State of Illinois.

Amendments: Any changes to this Agreement must be in writing and signed by both parties.

Signatures: This Agreement is signed on the date first above written.

Landlord Signature: John Doe ___________________________ Date: February 24, 2024

Tenant Signature: Jane Smith ___________________________ Date: February 24, 2024

Tenant Signature: Alex Johnson _________________________ Date: February 24, 2024"""

In [32]:
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

In [37]:
def basic_question_format(subject, question):
  return (
      USER_CHAT_TEMPLATE.format(
          prompt=f"What is the {subject[0]}?"
      )
      + MODEL_CHAT_TEMPLATE.format(prompt=subject[1])
      + USER_CHAT_TEMPLATE.format(prompt=question)
      + '<start_of_turn>model\n'
  )

# Inferencing

In [40]:
# basic lease summarization
subject = ["lease", example_lease]
primary_question = f"What are the key points of the {subject[0]}?"
fannout_questions = []
observations = model.generate(
    basic_question_format(subject, primary_question),
    device=device,
    output_len=max(100, int(len(subject[1])*.05))
)


In [41]:
observations

'Sure, here are the key points of the lease agreement:\n\n* The tenant pays the landlord $1,200 per month, due on the first day of each month.\n* The lease term is for 6 months, from March 1, 2024, to February 28, 2025.\n* The security deposit of $2,400 is due before moving in and will be returned within 30 days after the lease ends, minus any deductions.\n* The tenant is responsible for keeping the apartment clean and sanitary.\n* The'